# 데이터 수집 및 전처리

# 데이터 병합

## 데이터셋 목록 (출처, 데이터명, 컬럼명, 데이터시작지점, 최종지점, 수집주기 등) 

|활용|파일명|데이터|시작 시점|최종지점|수집주기|기준일|
|-|-|-|-|-|-|-|
|범죄 발생 건수 집계|crime_location.csv|2010|2020|1년|2021|
|<font color = 'gray'>사회적 요소 집계 <font/>|income_level_by_region.csv|2000|2019|1년|2021|
|<font color = 'gray'>사회적 요소 집계 <font/>|economic_activity_by_region.csv|2000|2020|1년|2021|
|<font color = 'gray'>사회적 요소 집계 <font/>|olice_officer_num_by_region.csv|2009|2017|1년|2021|
|인구적 요소|population_by_region.csv|1992|2020|1년|2021|
|인구적 요소|population_movement_by_region.csv|2001|2020|1년|2021|
|인구적 요소|외국인_거주자.csv|2003|2020|1년|2021|
|<font color = 'gray'>공간적 요소 집계<font/>|local_data.csv|2000|2020|1년|2021|
|<font color = 'gray'>공간적 요소 집계<font/>|universities_by_region.csv|2003|2020|1년|2021|
|기타 요인|시도_산업_조직형태별_사업체수_종사자수.csv|2000|2019|1년|2021|
|기타 요인|number_of_psychotropic_drug_requests_by_region.csv|2010|2020|1년|2021|





## 필요 라이브러리, 데이터 불러오기 및 경로 설정

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from matplotlib import font_manager,rc
import seaborn as sns
from sklearn.preprocessing import minmax_scale
from pyarrow import csv
import json

In [19]:
data_path ='../data/'
# Json 파일 읽어오기
def read_json(jsonPath ,mod = 'r',encoding ='utf-8'):
    with open(jsonPath, mod, encoding = encoding) as common:
        config = json.load(common)
    return config
config = read_json(data_path + "json/config.json")

## 공간적 요소 병합

In [19]:
local_path = config['data_path']+ 'csv/localData/cleaned/'

flag = True
for fileN in os.listdir(local_path):
    readed_file = csv.read_csv(local_path +fileN).to_pandas()
    readed_file = readed_file.dropna()
    tmp = pd.DataFrame(columns=['시도','년도',readed_file.columns[-1]])
    for city in readed_file.시도.unique():
        for year in readed_file[readed_file.시도 == city].연도.unique():
            tmp= tmp.append(pd.Series(
                [
                    city,
                    year,
                    readed_file[((readed_file.시도 == city) & (readed_file.연도== year))].iloc[:,-1].sum()
                    ] , index = tmp.columns)
                ,ignore_index= True)
    if flag :
        result = tmp.copy()
        flag = False
        continue
    
    result = pd.merge(result, tmp, left_on=['시도','년도'], right_on=['시도','년도'], how='left')
result = result.fillna(0) 
# 병합할 칼럼들 ( 유사 의미를 가지고 있는 칼럼 병합)
restrant = ['일반음식점','휴게음식점','관광식당']
bar = ['단란주점', '유흥주점', '외국인전용유흥음식점업']

result['음식점'] =result[restrant].sum(axis = 1)
result['유흥가'] = result[bar].sum(axis = 1)
#병합된 칼럼 제거
result = result.drop(restrant+bar,axis= 1)

# 단위개 개수이기 때문에 소수점 제거
result.iloc[:,1:] = result.iloc[:,1:].astype(int)

result.to_csv(config['data_path']+ 'csv/semicleaned/local.csv',encoding='utf-8-sig',index=False)

## 범죄 발생지 병합

In [28]:
# 범죄 발생지 병합
merged_df = pd.DataFrame()
flag = True
for fp in os.listdir(config['data_path']+ 'csv/crime_location'):     
    merged_csv = csv.read_csv(config['data_path']+ 'csv/crime_location/'+fp).to_pandas()
    merged_csv['시도'] = merged_csv['시도'].map(config['cities_mapping'])
    if flag :
        merged_df = merged_csv
        flag = False
    else:
        merged_df = pd.concat([merged_df,merged_csv], ignore_index=True)

# 데이터 병합
merged_df['폭행'] = merged_df['폭행'].add(merged_df['상해'])
merged_df['공갈'] = merged_df['공갈'].add(merged_df['협박'])
merged_df = merged_df.drop(['상해','협박'],axis=1)
merged_df = merged_df.rename(columns={'폭행':'폭행및상해','공갈':'공갈및협박'})



# -로 저장되어있는 값들 NA값으로 변경
def cleaning(x) :
  if str(x).find("-") != -1 :
    return str(x).replace("-",'')
  return x

for colName in merged_df.columns:
    merged_df[colName] = merged_df[colName].apply(cleaning)

# result.drop_duplicates()
merged_df.drop_duplicates().to_csv(config['data_path']+ 'csv/semicleaned/crime_location.csv',encoding='utf-8-sig',index=False)


## 모든 데이터 병합

### 제거한 데이터

- 실질적 일반 사람에게 피해가 없는 `범죄 파라미터` 제거
- 특정 대학의 수는 의미가 없다 판단하여 대학교 총계를 제외한 `대학수 제거`
- 성비를 이용한 인구수는 기타 데이터와 연계시 데이터 부족에 의하여 제거 `성별로 나누어진 데이터 제거`
- 시도간 전입,전출 칼럼을 보유하면 기다 전입 `전출 데이터`가 불필요하다 판단이 되어 제거

In [29]:
# 모든 데이터 병합
result = pd.DataFrame()
flag = True
for fp in os.listdir(config['data_path']+ 'csv/semicleaned'):
    
    merged_csv = csv.read_csv(config['data_path']+ 'csv/semicleaned/'+fp).to_pandas()
    merged_csv['시도'] = merged_csv['시도'].map(config['cities_mapping'])
    if flag :
        result = merged_csv
        flag = False
    else:
        result = pd.merge(
            result, merged_csv,
            left_on=['년도','시도'],
            right_on=['년도','시도'],
            how='left')
# 만약 중복되어 삽입된 행이 있다면 제거
result.drop_duplicates()
# 불필요하다 판단된 칼럼 제거
result = result.drop(config['dropCols'],axis = 1).drop_duplicates()
result.to_csv(config['data_path']+ 'csv/cleaned/crime_merged.csv',encoding='utf-8-sig',index=False)

## 범죄별 인덱싱

In [31]:
merged_df = csv.read_csv(config['data_path']+ 'csv/cleaned/crime_merged.csv').to_pandas()
merged_df.columns[2:15]

Index(['절도', '살인', '강도', '방화', '성폭력', '폭행및상해', '공갈및협박', '약취와 유인', '체포와 감금',
       '주거침입', '유기', '공무방해', '도주와범인은닉'],
      dtype='object')

In [53]:
merged_df = csv.read_csv(config['data_path']+ 'csv/cleaned/crime_merged.csv').to_pandas()
# 범죄 소계 데이터가 2002 ~ 2019까지 있기 떄문에 그 이외의 데이터 제거
merged_df = merged_df[(merged_df.년도 >=2002)& (merged_df.년도 <=2019)]


yearNcities_name= merged_df.columns[:2].to_list()
crimes = merged_df.columns[2:15].to_list()
indiventent_val_name = merged_df.columns[17:].to_list()
idx = 0
# 범죄 소계로 초기화

crime_Specification=pd.DataFrame([[idx,'범죄소계']],columns = ['idx','범죄명'])

result = merged_df[yearNcities_name + indiventent_val_name]
result['범죄수'] = merged_df[crimes].sum(axis = 1)
result['범죄종류'] = idx
result = result[yearNcities_name + ['범죄종류','범죄수'] + indiventent_val_name]
idx+=1

for crime_name in  crimes:
    for_Merge = merged_df[yearNcities_name + [crime_name] + indiventent_val_name]
    for_Merge = for_Merge.rename(columns={crime_name:'범죄수'})
    for_Merge['범죄종류'] = idx   

    crime_Specification = crime_Specification.append(pd.Series(
            [
                idx,
                crime_name
            ] , index = crime_Specification.columns)
            ,ignore_index= True)
    result =pd.concat(
        [
        result,
        for_Merge[yearNcities_name + ['범죄종류','범죄수'] + indiventent_val_name]
        ],
        ignore_index=True)
    idx+=1
result.to_csv(config['data_path']+ 'csv/cleaned/crime_indexed.csv',encoding='utf-8-sig',index=False)
crime_Specification.to_csv(config['data_path']+ 'csv/cleaned/crime_Specification.csv',encoding='utf-8-sig',index=False)


<ipython-input-53-65bc958489c4>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['범죄수'] = merged_df[crimes].sum(axis = 1)
<ipython-input-53-65bc958489c4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['범죄종류'] = idx
